<a href="https://colab.research.google.com/github/Lukehsu1999/Academic-and-Formality-Rewriter/blob/main/evaluation/Classify_Model_Outputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
!pip install -q transformers

import tensorflow as tf
from tensorflow import keras
from transformers import TFBertForSequenceClassification
from transformers import BertTokenizer

In [ ]:
tokenizer=BertTokenizer.from_pretrained('bert-base-cased')

# Settings

In [ ]:
task = "Formal" # "Formal" "Academic"
set_path = "EM_test"
test_code = "EM"
test_full_name = "Entertainment_Music"  #"Entertainment_Music" "Family_Relationships"
path_to_save_dict = "/content/drive/MyDrive/CityU/FYP/Evaluation/Classification_Results/"+task+"_Classification/"+test_code+"_results.json"

Path to classifiers: <br>
Formality Classifier: <br>
/content/drive/MyDrive/CityU/FYP/Models/Formality_Classifier/Epoch2Jun15
<br>
<br>
Academic Classifier: <br>
/content/drive/MyDrive/CityU/FYP/Models/Academic_Classifier/AcademicEpoch2June14

In [ ]:
path_to_formality_classifier = "/content/drive/MyDrive/CityU/FYP/Models/Formality_Classifier/Epoch2Jun15"
path_to_academic_classifier = "/content/drive/MyDrive/CityU/FYP/Models/Academic_Classifier/AcademicEpoch2June14"

In [ ]:
if task == "Academic":
  print("Academic Classifier")
  path_to_model_weight= path_to_academic_classifier
else:
  print("Formality Classifier")
  path_to_model_weight= path_to_formality_classifier


demo_model=TFBertForSequenceClassification.from_pretrained('bert-base-cased')
demo_model.load_weights(path_to_model_weight)

Formality Classifier


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def getPrediction(sentence,thismodel):
  inputs=tokenizer(sentence,return_tensors="tf")
  outputs=thismodel(inputs,return_dict=True)
  #print(outputs)
  #print(outputs.loss)
  #print(outputs.logits)
  logits=outputs.logits
  probs = tf.nn.softmax(logits)
  # print(probs)
  prediction = tf.math.argmax(probs, axis=1)
  # print(prediction)
  #print(f"probs {probs}")
  if prediction[0]==1:
    #print("Formal")
    return 1, str(probs)
  else:
    #print("Informal")
    return 0, str(probs)

In [ ]:
def get_formal_count(path_to_model_output):
  file = open(path_to_model_output, "r")
  lines = file.readlines()
  total_line_cnt = len(lines)
  print("Total lines: "+str(total_line_cnt))
  formal_cnt = 0
  result_lst = []
  for line in lines:
    res, certainty = getPrediction(line, demo_model)
    formal_cnt += res
    result_lst.append(certainty)
  print("formal lines: "+str(formal_cnt))
  print("formal percentage: "+str(formal_cnt/total_line_cnt))
  return result_lst

Choose Corpus Here

In [ ]:
model_output_path_dict = {
    "EMFR":"/content/drive/MyDrive/CityU/FYP/Model_Outputs/"+set_path+"/EMFR_model_3pochs_"+test_code+"_test",
    "EM":"/content/drive/MyDrive/CityU/FYP/Model_Outputs/"+set_path+"/EM_model_3pochs_"+test_code+"_test",
    "FR":"/content/drive/MyDrive/CityU/FYP/Model_Outputs/"+set_path+"/FR_model_3pochs_"+test_code+"_test",
    "nmtbaseline":"/content/drive/MyDrive/CityU/FYP/Model_Outputs/"+set_path+"/formal.nmt_baseline",
    "nmtcombined":"/content/drive/MyDrive/CityU/FYP/Model_Outputs/"+set_path+"/formal.nmt_combined",
    "nmtcopy":"/content/drive/MyDrive/CityU/FYP/Model_Outputs/"+set_path+"/formal.nmt_copy",
    "pbmt":"/content/drive/MyDrive/CityU/FYP/Model_Outputs/"+set_path+"/formal.pbmt",
    "rulebased":"/content/drive/MyDrive/CityU/FYP/Model_Outputs/"+set_path+"/formal.rule_based",
    "ref0":"/content/drive/MyDrive/CityU/FYP/Model_Outputs/"+test_code+"_test/references/formal.ref0",
    "ref1":"/content/drive/MyDrive/CityU/FYP/Model_Outputs/"+test_code+"_test/references/formal.ref1",
    "ref2":"/content/drive/MyDrive/CityU/FYP/Model_Outputs/"+test_code+"_test/references/formal.ref2",
    "ref3":"/content/drive/MyDrive/CityU/FYP/Model_Outputs/"+test_code+"_test/references/formal.ref3",
}

In [ ]:
result_dict = {}

In [ ]:
for k in model_output_path_dict.keys():
  print(k)
  path = model_output_path_dict[k]
  res_lst = get_formal_count(path)
  result_dict[k] = res_lst

Save

In [ ]:
import json

# json save
with open(path_to_save_dict, 'w')as fp:
  json.dump(result_dict, fp)